In [ ]:
## tidied up scripts -- Mar 2017

#### Using NYT's CRF ingredient tagger as a means to 'clean' up the ingredients a little more...  

Refs.  
- https://github.com/hengrumay/ingredient-phrase-tagger || https://github.com/NYTimes/ingredient-phrase-tagger
- http://open.blogs.nytimes.com/2015/04/09/extracting-structured-data-from-recipes-using-conditional-random-fields/?_r=1
- https://rajmak.wordpress.com/2016/02/19/structuring-text-using-conditional-random-field-crf-tagging-recipe-ingredient-phrases/
   
-- ref prior code exploration: `Re-coding_parseIngredients_run_CRFtaggerNsomecleanup_v0.ipynb`   

#### However it is still not ideal -- some additional tweaks needed to improve added...   
-- `next script ref Retrieve_CRFtaggedCleanedFilesNtidy4use_v0.ipynb`

####    





In [1]:
from __future__ import print_function, division

import pandas as pd
import numpy as np

import pickle

In [2]:
filepath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/bbc_goodfood_recipes_pickles/'

In [ ]:
DF = pd.read_pickle(filepath+'BBCgoodfood_10081recipesNlinks.pkl')

### Some tidying before using CRF++

In [ ]:
import re
import string

In [ ]:
def replaceUnits(s):
    
    metricNtype_unitlabels = {
        'tsp'   : 'teaspoon ',
        'tbsp'  : 'tablespoon ',
        'g'     : 'gram ',
        'kg'    : 'kilogram ',
        'kg/'   : 'kilogram/ ',
        'ml'    : 'milliliter ',
        'l'     : 'liter ',
    ' kilogram' : 'kilogram',
        ' cm'   : 'cm', 
        
        ' fl oz': '_fl_oz ',
        
     'rounded '  : '',#rounded*',
        'round'  : '',
    'handful of ': '1 handful ',
    'small handful '   : '1 handful ',
    'a handful '   : '1 handful ',    
        'heaped'  : '',
        'a few'   : '2-3',#'a-few',

        'can'    : '', #canned ',
        'cans'   : '', #canned ',
        'pot'    : '',#'potted '
        'pots'   : '',#'potted '
        'tub'    : '',
        'tubs'   : '',        
        'jar'    : '',
        'pack'   : '',
        'bag'    : '',
        'sachet' : '',
       'sachets' : 'sachets of ',
        'punnet' : '',
        'tub'    : '',
        'pkt'    : '', 
     'block of ' : '',
        
       #'bunch' : '1*bunch',
        
        }

    for unit, label in metricNtype_unitlabels.items():
        #print (unit, label)
        #s = s.replace(unit, label)
        s = re.sub(r'(?<=[\d\s*])%s(\s)'%(unit), label, s)
        
        s = s.replace('kg/', 'kilogram /')
        s = s.replace('g/', 'gram /')
    #         if 'bunch' in s.split(' '):
    #             s = s.replace('bunch ', '') 
    #             #s = s+' bunch'
        if 'few drops' in s:
            s=s.replace('few drops','') +', few drops'
            
        if 'good thumb-size' in s:
            s=s.replace('good ','')
        if 'good pinch ' in s:
            s=s.replace('good ','')
        
        s = s.replace(' x ', '*')
        s = s.replace('*x ','*')

    return s


In [ ]:
def recodeFractionStrs(s):    
    
#     fractionWds = {
        
#         'half a' : str(1/2),
#         'half ' : str(1/2),
#         '½ a '  : str(1/2),
#         '0.5*a ': str(1/2),
#         'a quarter ' : str(1/4),
#         ' liter'    : 'liter'

#         }
    
#     for wfrac, facSTR in fractionWds.items():
#         s = s.replace(wfrac ,facSTR+'*')
        
        
    fractionStrs = {
           '1⁄8': str(1/8),
           '3⁄8': str(3/8),
           '5⁄8': str(5/8),
           '7⁄8': str(7/8),
           '1⁄6': str(1/6),
           '5⁄6': str(5/6),
           '1⁄5': str(1/5),
           '2⁄5': str(2/5),
           '3⁄5': str(3/5),
           '4⁄5': str(4/5),
           '1⁄4': str(1/4),          
           '3⁄4': str(3/4),
           '1⁄3': str(1/3),
           '2⁄3': str(2/3),
           '1⁄2': str(1/2),
            
            '⅛': str(1/8),
            '⅜': str(3/8),
            '⅝': str(5/8),
            '⅞': str(7/8),
            '⅙': str(1/6),
            '⅚': str(5/6),
            '⅕': str(1/5),
            '⅖': str(2/5),
            '⅗': str(3/5),
            '⅘': str(4/5),
            '¼': str(1/4),
            '¾': str(3/4),
            '⅓': str(1/3),
            '⅔': str(2/3),
            '½': str(1/2),        
    }


    for ufrac, fracStr in fractionStrs.items():
        #print (unit, label)
        #s = s.replace(unit, label)
        s = re.sub(r'(?<=[\d\s*])%s(\s)'%(ufrac), '+'+fracStr+' ', s)
        #s = re.sub(r'^(\s*)%s(\s*)'%('½'), '1/2*', DF.ingred[32][-1])
        s = re.sub(r'^(\s*)%s(\s*)'%(ufrac), fracStr+'*', s)
        
        s = s.replace(ufrac+'-','+'+fracStr+'to')
        s = s.replace(ufrac,'+'+fracStr+' ')
        s = s.replace(' '+ufrac,'+'+fracStr+' ')
        s = s.replace(' '+'+'+fracStr+' ','+'+fracStr+' ')
    
    s = s.replace('half a',str(1/2)+'*')
    s = s.replace('half ',str(1/2)+'*')
    s = s.replace('½ a ',str(1/2)+'*')
    s = s.replace('0.5*a ',str(1/2)+'*')
    
    s = s.replace('a quarter',str(1/4)+'*')
    s = s.replace(' liter','liter')
    s = s.replace('0.5* glass','0.5*glass')
    

    return s

In [ ]:
def renameSomeIngredients(s):

    IngdName = {
        'spring onion' : 'spring-onion',
        'bok choi'     : 'bok-choi',
        'pak choi'     : 'pak-choi',
        'nam pla'      : 'nam-pla',
        'kecap manis'  : 'kecap-manis',
        'kaffir lime'  : 'Kaffir lime',
        'egg white'    : 'egg-white',
        'egg yolk'    : 'egg-yolk',
        'yolk'        : 'egg-yolk',
        
        'dark chocolate' : 'dark-chocolate',
        
        'milliliter coconut milk' : 'milliliter, coconut milk',
        
        'chilli deseeded'    : 'chilli, deseeded',
        'pepper deseeded'    : 'pepper, deseeded',
        
        'finely grated zest' : 'the zest of',
        'grated zest'        : 'the zest of',
                   'zest '   :   'the zest of ',
        'juice 1 lime'      : '1 lime, juiced', 
        'juice of 1 lime'   : '1 lime, juiced',
        'juice 2 limes'     : '2 limes, juiced ',
        #'and juice 1'   : 'and juice of 1',
        'juice of and 1 lime'     :  '1 lime and its juice',
        'zest and juice'   : '#zest&juice',
        'juice of+'          : 'juice of +',
        '1 lemon and 1 orange': '1 each of lemon&orange',
        

        'Cointreau or Grand Marnier'  : 'Cointreau-or-Grand-Marnier',
        
        'iceberg lettuce'    : 'iceberg-lettuce',
        
        'ancho or other large dried chilli, ' : 'dried ancho chilli, (',
        
        'tomatoes, whole or chopped ' : 'whole or chopped tomatoes (',
        'tomato in oil'    : 'tomato',
        'sundried tomato'  : 'sundried-tomato,',
        'plum tomato '   : 'plum tomato, ',
        'fresh tomato pasta sauce' : 'fresh tomato pasta-sauce,',
        
        'pickled red cabbage'   : 'red cabbage, pickled,',
        'artichoke hearts'    :    'artichoke-hearts,',
        'cooked rice'    : 'rice, cooked,' ,
        
        'seeds scraped from 1 vanilla pod or ' : '', #1 vanilla pod',
        'seeds+0.5 vanilla pod or'             : '',
        
        'cinnamon, fennel seeds, ground cumin' : 'cinnamon&fennel-seeds&ground-cumin',
        'chickpeas or'  : 'chickpeas, or',
        
        'grated emmental or gruyère'  :   'emmental/gruyère, grated ',
        'tortilla chips or homemade tortilla strips ' : 'tortilla chips or strips ',
        
        'full-fat'  : '',
        'butter '    : 'butter, ',
        
        'sprigs'    : 'sprigs,' ,
        
        'good-quality,'  : '',
        'firm,'   : '',
        'fat, fresh'  : 'large fresh',
        
        '*cucumber'   : '* cucumber,',
        '*garlic'     : '* garlic,',
        '*teaspoon'   : ' * teaspoon',
        '*table'   : ' * table',
        
    }
    
    for listwds, name in IngdName.items():
        s = s.replace(listwds,name)
    
    return s

In [ ]:
DF['ingred00'] = DF.ingred.apply(lambda x: [renameSomeIngredients(i) for i in x])

In [ ]:
DF['ingred0'] = DF.ingred00.apply(lambda x: [recodeFractionStrs(i) for i in x])

In [ ]:
# DF['ingred1'] = DF.ingred.map(lambda x: [replaceUnits(n) for n in x])
DF['ingred1'] = DF.ingred0.map(lambda x: [replaceUnits(n) for n in x])

In [ ]:
DF

### CRF++ USAGE HERE 

takes a while to run...

In [3]:
from __future__ import print_function

import sys
import os
import tempfile

# from ingredient_phrase_tagger.training import utils

import sklearn
import pycrfsuite

In [4]:
## MUST change dir to run following scripts! Do Once...
os.chdir(os.path.dirname('./ingredient-phrase-tagger-master/'))
# !pwd

from ingredient_phrase_tagger.training import utils

In [5]:
# print(os.getcwd())
# os.getcwd() 
Modelpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes/ingredient-phrase-tagger-master/'
Folderpath = '/Users/hrm/Documents/Dropbox/DSrelated/Metis/recipes'

In [6]:
crf_filespath = Folderpath +'/crf_tagger_files/'

In [ ]:
filename = [crf_filespath + 'bbcgdfd_'+str(i)+'.txt' for i in list(DF.index)]

In [ ]:
###################################################################################

In [ ]:
## Convert list of ingredients to textfiles using write
for f in range(len(filename)):
    #print(f)
    with open(filename[f], 'w') as outfile:
        outfile.write("\n".join(DF.ingred1[f]))
        #outfile.write("\n".join(DF.ingred[f]))

In [ ]:
for f in range(len(filename)):
    
    with open(filename[f], 'r') as infile, open('tmp.txt', 'w') as outfile:
        outfile.write(utils.export_data(infile.readlines()))

    modelFilename = Modelpath +'/tmp/model_file'
    tmpFile = Modelpath +'/tmp.txt'
    outFile = crf_filespath +'bbcgdfd_'+str(f)+'o.txt'

    os.system("crf_test -v1 -m %s %s > %s" % (modelFilename,tmpFile,outFile))
    os.system("rm %s" % tmpFile)

In [ ]:
###################################################################################

### Check -- readfile

In [7]:
# readfile = crf_filespath +'bbcgdfd_'+str(4)+'o.txt'
outtest = utils.import_data(open(crf_filespath +'bbcgdfd_'+str(8888)+'o.txt')) 

In [8]:
df0 = pd.DataFrame(outtest).drop('display',axis=1)
df0

,comment,input,name,other,qty,unit
0,NaN,500milliliter double cream,double cream,NaN,500milliliter,NaN
1,broken into chunks,"300gram dark-chocolate, broken into chunks",dark-chocolate,",",300gram,NaN
2,sifted,"2 tablespoon icing sugar, sifted",icing sugar,",",2,tablespoon
3,NaN,a few sachets of sherbet,few sachets of sherbet,NaN,a,NaN
4,NaN,200milliliter whipping cream,whipping cream,NaN,200milliliter,NaN
5,NaN,"50milliliter Prosecco ,",Prosecco,",",50milliliter,NaN
6,sifted,"2 tablespoon icing sugar, sifted",icing sugar,",",2,tablespoon


In [ ]:
### Requires further tidying up of CRF outputs... e.g. qty wrt metric units
# REF Retrieve_CRFtaggedCleanedFilesNtidy4use_v0